<a href="https://colab.research.google.com/github/rashad101/merge-llms/blob/main/LLM_merge-zephyr-7b-beta-passthrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .
import torch

Cloning into 'mergekit'...
remote: Enumerating objects: 1033, done.
remote: Counting objects: 100% (573/573), done.
remote: Compressing objects: 100% (217/217), done.
remote: Total 1033 (delta 458), reused 397 (delta 351), pack-reused 460
Receiving objects: 100% (1033/1033), 270.47 KiB | 1.45 MiB/s, done.
Resolving deltas: 100% (698/698), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.0 MB/s eta 0:

In [2]:
torch.cuda.is_available()

True

In [3]:
import yaml

MODEL_NAME = "zephyr-7b-beta-passthrough"
yaml_config = """
slices:
  - sources:
    - model: HuggingFaceH4/zephyr-7b-beta
      layer_range: [0, 32]
  - sources:
    - model: mlabonne/NeuralHermes-2.5-Mistral-7B
      layer_range: [16, 32]
base_model: HuggingFaceH4/zephyr-7b-beta
merge_method: passthrough
dtype: bfloat16

"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

In [4]:
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle --trust-remote-code


Streaming output truncated to the last 5000 lines.


model-00003-of-00003.safetensors:   0% 21.0M/4.54G [00:00<01:42, 43.9MB/s]

model-00002-of-00003.safetensors:   0% 10.5M/5.00G [00:00<04:04, 20.4MB/s]



model-00001-of-00003.safetensors:   1% 31.5M/4.94G [00:00<01:53, 43.4MB/s]




model-00003-of-00003.safetensors:   1% 31.5M/4.54G [00:00<01:44, 43.3MB/s]

model-00002-of-00003.safetensors:   0% 21.0M/5.00G [00:00<02:50, 29.1MB/s]



model-00001-of-00003.safetensors:   1% 41.9M/4.94G [00:00<01:51, 43.8MB/s]




model-00003-of-00003.safetensors:   1% 41.9M/4.54G [00:00<01:43, 43.3MB/s]

model-00002-of-00003.safetensors:   1% 31.5M/5.00G [00:01<02:24, 34.3MB/s]



model-00001-of-00003.safetensors:   1% 52.4M/4.94G [00:01<01:52, 43.4MB/s]




model-00003-of-00003.safetensors:   1% 52.4M/4.54G [00:01<01:49, 41.0MB/s]

model-00002-of-00003.safetensors:   1% 41.9M/5.00G [00:01<02:19, 35.6MB/s]



model-00001-of-00003.safetensors:   1% 62.9M/4.94G [00:01<01:56, 41.8MB/s]




model-00003-of-

In [6]:
!pip install -qU huggingface_hub

from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template

username = "rony"

template_text = """
---
license: mit
tags:
- merge
- mergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

```yaml
{{- yaml_config -}}
```
"""

# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Get list of models from config
data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')

In [8]:
from google.colab import userdata
from huggingface_hub import HfApi

username = "rony"

# Defined in the secrets tab in Google Colab
api = HfApi(token=userdata.get("hf_token"))

#api.create_repo(
#    repo_id=f"{username}/{MODEL_NAME}",
#    repo_type="model"
#)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)


CommitInfo(commit_url='https://huggingface.co/rony/zephyr-7b-beta-passthrough/commit/268c6954863869f54c38baec67e8cd45a08e184e', commit_message='Upload folder using huggingface_hub', commit_description='', oid='268c6954863869f54c38baec67e8cd45a08e184e', pr_url=None, pr_revision=None, pr_num=None)